In [23]:
import os
from typing import Optional
from serpapi import SerpApiClient
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from dotenv import load_dotenv
load_dotenv()


# (Your Pydantic models FlightsInput and FlightsInputSchema remain the same)
class FlightsInput(BaseModel):
    departure_airport: Optional[str] = Field(description='Departure airport code (IATA)')
    arrival_airport: Optional[str] = Field(description='Arrival airport code (IATA)')
    outbound_date: Optional[str] = Field(description='Parameter defines the outbound date. The format is YYYY-MM-DD. e.g. 2024-06-22')
    return_date: Optional[str] = Field(description='Parameter defines the return date. The format is YYYY-MM-DD. e.g. 2024-06-28')
    adults: Optional[int] = Field(1, description='Parameter defines the number of adults. Default to 1.')
    children: Optional[int] = Field(0, description='Parameter defines the number of children. Default to 0.')
    infants_in_seat: Optional[int] = Field(0, description='Parameter defines the number of infants in seat. Default to 0.')
    infants_on_lap: Optional[int] = Field(0, description='Parameter defines the number of infants on lap. Default to 0.')

class FlightsInputSchema(BaseModel):
    params: FlightsInput


@tool(args_schema=FlightsInputSchema)
def flights_finder(params: FlightsInput):
    '''
    Find flights using the Google Flights engine.

    Returns:
        dict: Flight search results.
    '''

    api_params = {
        'api_key': os.getenv('SERPAPI_API_KEY'),
        'engine': 'google_flights',
        'hl': 'en',
        'gl': 'us',
        'departure_id': params.departure_airport,
        'arrival_id': params.arrival_airport,
        'outbound_date': params.outbound_date,
        'return_date': params.return_date,
        'currency': 'USD',
        'adults': params.adults,
        'infants_in_seat': params.infants_in_seat,
        'stops': '1',
        'infants_on_lap': params.infants_on_lap,
        'children': params.children
    }

    try:
        client = SerpApiClient(api_params)
        search_results = client.get_dict()

        # --- NEW: Check for an explicit error message from the API ---
        if 'error' in search_results:
            return f"An API error occurred: {search_results['error']}"

        # --- IMPROVED: Check for the key before trying to access it ---
        if 'best_flights' in search_results:
            return search_results['best_flights']
        else:
            # Provide a clear, helpful message if no flights are found
            return "No flights were found for the specified criteria. Please try different dates or airports."

    except Exception as e:
        # Fallback for other unexpected errors
        return f"An unexpected error occurred: {str(e)}"

In [24]:

flights_finder({
  "params": {
    "departure_airport": "LHR",
    "arrival_airport": "DXB",
    "outbound_date": "2026-04-10",
    "return_date": "2026-04-24",
    "adults": 2,
    "children": 1,
    "infants_in_seat": 1
  }
})

[{'flights': [{'departure_airport': {'name': 'Heathrow Airport',
     'id': 'LHR',
     'time': '2026-04-10 21:30'},
    'arrival_airport': {'name': 'Dubai International Airport',
     'id': 'DXB',
     'time': '2026-04-11 07:35'},
    'duration': 425,
    'airplane': 'Boeing 787-10',
    'airline': 'British Airways',
    'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/BA.png',
    'travel_class': 'Economy',
    'flight_number': 'BA 109',
    'legroom': '31 in',
    'extensions': ['Average legroom (31 in)',
     'Wi-Fi for a fee',
     'In-seat power & USB outlets',
     'On-demand video',
     'Carbon emissions estimate: 1174 kg'],
    'overnight': True}],
  'total_duration': 425,
  'carbon_emissions': {'this_flight': 1175000,
   'typical_for_this_route': 1537000,
   'difference_percent': -24},
  'price': 2395,
  'type': 'Round trip',
  'airline_logo': 'https://www.gstatic.com/flights/airline_logos/70px/BA.png',
  'departure_token': 'WyJDalJJWlUxcGFXRTBUa042Y2pSQlE

In [29]:
import os
from typing import Optional

# --- Import the specific client from the serpapi library ---
from serpapi import SerpApiClient
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool

class HotelsInput(BaseModel):
    q: str = Field(description='Location of the hotel (e.g., "Miami Beach, Florida")')
    check_in_date: str = Field(description='Check-in date. The format is YYYY-MM-DD. e.g. 2025-11-20')
    check_out_date: str = Field(description='Check-out date. The format is YYYY-MM-DD. e.g. 2025-11-25')
    # --- Changed default to a string for robustness ---
    sort_by: Optional[str] = Field("8", description='Parameter for sorting results. Default is "8" for highest rating.')
    adults: Optional[int] = Field(1, description='Number of adults. Default to 1.')
    children: Optional[int] = Field(0, description='Number of children. Default to 0.')
    rooms: Optional[int] = Field(1, description='Number of rooms. Default to 1.')
    hotel_class: Optional[str] = Field(
        None, description='Filter by hotel class (e.g., "2,3,4" for 2, 3, and 4-star hotels).')

class HotelsInputSchema(BaseModel):
    params: HotelsInput

@tool(args_schema=HotelsInputSchema)
def hotels_finder(params: HotelsInput):
    '''
    Find hotels using the Google Hotels engine.

    Returns:
        dict or str: Hotel search results or an error message.
    '''
    # --- Renamed to api_params to avoid confusion with the input 'params' object ---
    api_params = {
        'api_key': os.environ.get('SERPAPI_API_KEY'),
        'engine': 'google_hotels',
        'hl': 'en',
        'gl': 'in',
        'q': params.q,
        'check_in_date': params.check_in_date,
        'check_out_date': params.check_out_date,
        'currency': 'INR',
        'adults': params.adults,
        'children': params.children,
        'rooms': params.rooms,
        'sort_by': params.sort_by,
        'hotel_class': params.hotel_class
    }

    try:
        # --- Use the modern, class-based method for the API call ---
        client = SerpApiClient(api_params)
        search_results = client.get_dict()

        # --- 1. Check for an explicit error message from the API ---
        if 'error' in search_results:
            return f"An API error occurred: {search_results['error']}"

        # --- 2. Check for the 'properties' key before trying to access it ---
        if 'properties' in search_results and search_results['properties']:
            # Return the top 5 results if they exist
            return search_results['properties'][:5]
        else:
            # --- 3. Provide a clear, helpful message if no hotels are found ---
            return "No hotels were found for the specified criteria. Please try different dates or locations."

    except Exception as e:
        # --- 4. Fallback for other unexpected errors (e.g., network issues) ---
        return f"An unexpected error occurred: {str(e)}"


In [30]:
hotels_finder({
  "params": {
    "q": "Eiffel Tower, Paris, France",
    "check_in_date": "2025-11-10",
    "check_out_date": "2025-11-15",
    "adults": 2
  }
})

[{'type': 'hotel',
  'name': 'Hôtel Tourisme Avenue',
  'description': 'Relaxed hotel offering bright rooms with free Wi-Fi, plus a terrace with seating.',
  'link': 'https://www.hoteltourismeavenue.com/',
  'property_token': 'ChgI5PP_6eWyk5b8ARoLL2cvMXRmX2dxdjIQAQ',
  'serpapi_property_details_link': 'https://serpapi.com/search.json?adults=2&check_in_date=2025-11-10&check_out_date=2025-11-15&children=0&currency=INR&engine=google_hotels&gl=in&hl=en&property_token=ChgI5PP_6eWyk5b8ARoLL2cvMXRmX2dxdjIQAQ&q=Eiffel+Tower%2C+Paris%2C+France&sort_by=8',
  'gps_coordinates': {'latitude': 48.8496072, 'longitude': 2.2983252},
  'check_in_time': '3:00\u202fPM',
  'check_out_time': '12:00\u202fPM',
  'rate_per_night': {'lowest': '₹18,079',
   'extracted_lowest': 18079,
   'before_taxes_fees': '₹15,014',
   'extracted_before_taxes_fees': 15014},
  'total_rate': {'lowest': '₹90,396',
   'extracted_lowest': 90396,
   'before_taxes_fees': '₹75,068',
   'extracted_before_taxes_fees': 75068},
  'nearby_

In [37]:
import os
from typing import TypedDict, Annotated, Optional, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
# --- Import ChatGroq instead of ChatOpenAI ---
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
# --- Import ToolNode and remove ToolExecutor ---
from langgraph.prebuilt import ToolNode
from serpapi import SerpApiClient
from dotenv import load_dotenv
load_dotenv()
from tools import flights_finder as ff, hotel_finder as hf


In [38]:
tools = [ff, hf]

# --- Define the LLM using Groq ---
model = ChatGroq(temperature=0, model_name="llama3-70b-8192")
model = model.bind_tools(tools)

ValueError: Unsupported function

<module 'tools.flights_finder' from 'c:\\Users\\r\\OneDrive\\Desktop\\ai trip planner\\tools\\flights_finder.py'>

Functions must be passed in as Dict, pydantic.BaseModel, or Callable. If they're a dict they must either be in OpenAI function format or valid JSON schema with top-level 'title' and 'description' keys.